# GarmentIQ Classification Quick Start

In [ ]:
import garmentiq as giq
from garmentiq.classification.model_definition import tinyViT
from garmentiq.classification.utils import CachedDataset

In [ ]:
# Download test data and a pretrained model
!mkdir -p /app/working/examples/classification_quick_start/models

!curl -L -o /app/working/examples/classification_quick_start/garmentiq-classification-set-nordstrom-and-myntra.zip \
  https://www.kaggle.com/api/v1/datasets/download/lygitdata/garmentiq-classification-set-nordstrom-and-myntra

!wget -q -O /app/working/examples/classification_quick_start/models/tiny_vit.pt \
    https://huggingface.co/lygitdata/garmentiq/resolve/main/tiny_vit.pt

In [ ]:
# Prepare test data garmentiq-classification-set-nordstrom-and-myntra.zip
DATA = giq.classification.train_test_split(
    output_dir="/app/working/examples/classification_quick_start/data",
    metadata_csv="metadata.csv",
    label_column="garment",
    train_zip_dir="/app/working/examples/classification_quick_start/garmentiq-classification-set-nordstrom-and-myntra.zip",
    test_size=0.15,
    verbose=True
)

test_images, test_labels, _ = giq.classification.load_data(
    df=DATA["test_metadata"],
    img_dir=DATA["test_images"],
    label_column="garment",
    resize_dim=(120, 184),
    normalize_mean=[0.8047, 0.7808, 0.7769],
    normalize_std=[0.2957, 0.3077, 0.3081]
)

In [ ]:
# Load the pretrained model
classifier = giq.classification.load_model(
    model_path="/app/working/examples/classification_quick_start/models/tiny_vit.pt",
    model_class=tinyViT,
    model_args={"num_classes": 9, "img_size": (120, 184), "patch_size": 6}
)

In [ ]:
# Fit the model on the whole test data
giq.classification.test_pytorch_nn(
    model_path="/app/working/examples/classification_quick_start/models/tiny_vit.pt",
    model_class=tinyViT,
    model_args={"num_classes": 9, "img_size": (120, 184), "patch_size": 6},
    dataset_class=CachedDataset,
    dataset_args={
        "raw_labels": DATA["test_metadata"]["garment"],
        "cached_images": test_images,
        "cached_labels": test_labels,
    },
    param={"batch_size": 64},
)

In [ ]:
# Fit the model on a single image
img_to_test = DATA['test_metadata']['filename'][88]

pred_label, pred_prob = giq.classification.predict(
    model=classifier,
    image_path=f"/app/working/examples/classification_quick_start/data/test/images/{img_to_test}",
    classes=DATA['test_metadata']['garment'].unique().tolist(),
    resize_dim=(120, 184),
    normalize_mean=[0.8047, 0.7808, 0.7769],
    normalize_std=[0.2957, 0.3077, 0.3081]
)

print(
    "True label: ", img_to_test,
    "\nPredicted label: ", pred_label,
    "\nPredicted Probabilities: ", pred_prob
)